# Import packages

In [462]:
from semanticscholar import SemanticScholar
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import json

import config_keys

import random
import time
import csv
import regex as re

from pandas import DataFrame, read_csv, option_context, Series, crosstab

import helpers

In [227]:
def print_all(text):
    with option_context('display.max_rows', None, 'display.max_columns', None):  
        # more options can be specified also
        print(text)

In [92]:
def write_csv(data, filename):
    with open(filename, 'w', encoding='UTF8') as f:
        # create the csv writer
        writer = csv.writer(f)
        # write a row to the csv file
        writer.writerow(data)

# First experiment

In [2]:
sch = SemanticScholar()
results = sch.search_paper('+"gender bias"', fields=['title','year', 'authors', 'publicationTypes', 'citationCount', 'url', 'fieldsOfStudy', 's2FieldsOfStudy', 'journal'])
print(f'{results.total} results.', f'First occurrence: {results[0].title}.')

2366962 results. First occurrence: Gender Bias in Machine Translation.


In [49]:
data = []
for result in results[0:10]:
    for author in result['authors']:
        data.append({'Title': result['title'], 
                    'Year': result['year'],
                    'Author': author['name'],
                    'PublicationType': result['publicationTypes'], 
                    'CitationCount': result['citationCount'],
                    'FieldsOfStudy': result['fieldsOfStudy'],
                    'FieldsOfStudy_s2': result['s2FieldsOfStudy'],
                    'Journal': result['journal'],
                    'URL': result['url']})

In [70]:
results[0]

{'paperId': '551f36edf960d196f8499bf5db37537dd6433318', 'url': 'https://www.semanticscholar.org/paper/551f36edf960d196f8499bf5db37537dd6433318', 'title': 'Stag Parties Linger: Continued Gender Bias in a Female-Rich Scientific Discipline', 'year': 2012, 'citationCount': 95, 'fieldsOfStudy': ['Medicine'], 's2FieldsOfStudy': [{'category': 'Medicine', 'source': 'external'}, {'category': 'Psychology', 'source': 's2-fos-model'}], 'publicationTypes': ['JournalArticle'], 'journal': {'name': 'PLoS ONE', 'volume': '7'}, 'authors': [{'authorId': '3900551', 'name': 'L. Isbell'}, {'authorId': '6896193', 'name': 'T. Young'}, {'authorId': '7032312', 'name': 'A. Harcourt'}]}

In [41]:
type(data[0]['Journal'])

dict

In [5]:
len(results)

100

In [61]:
df = DataFrame(data, columns = ['Title', 'Year', 'Author', 'PublicationType', 'CitationCount', 'URL', 'FieldsOfStudy', 'FieldsOfStudy_s2', 'Journal'])
df[['FieldsOfStudy', 'Journal']].head(10)

,FieldsOfStudy,Journal
0,[Computer Science],{'name': 'Transactions of the Association for ...
1,[Computer Science],{'name': 'Transactions of the Association for ...
2,[Computer Science],{'name': 'Transactions of the Association for ...
3,[Computer Science],{'name': 'Transactions of the Association for ...
4,[Computer Science],{'name': 'Transactions of the Association for ...
5,[Computer Science],{'pages': '15-20'}
6,[Computer Science],{'pages': '15-20'}
7,[Computer Science],{'pages': '15-20'}
8,[Computer Science],{'pages': '15-20'}
9,[Computer Science],{'pages': '15-20'}


In [64]:
df.Journal.iloc[0].keys()

dict_keys(['name', 'pages', 'volume'])

In [ ]:
df['JournalTitle'] = df['Journal'].apply(lambda x: x.get('name') if 'name' in x.keys() else None)
df['JournalTitle']

In [ ]:
l = ['name']
journal = [x.get('name') if 'name' in x.keys() else 'F' for x in df['Journal'] ] #.apply(lambda x: x['name'])
journal

## Data 2012-2023

In [3]:
data_authors = []
df_authors = DataFrame(columns = ['Title', 
    'Year', 
    'Author', 
    'PublicationType', 
    'CitationCount', 
    'FieldsOfStudy', 
    'FieldsOfStudy_s2', 
    'Journal',
    'URL'])

sch = SemanticScholar()
for year in range(2012, 2024):
    results = sch.search_paper('+"gender bias"', 
        fields=['title','year', 'authors', 'publicationTypes', 'citationCount', 'fieldsOfStudy', 's2FieldsOfStudy', 'journal', 'url'],
        year=year)
    print(len(results))
    for result in results:
        for author in result['authors']:
            data_authors.append({'Title': result['title'], 
                    'Year': result['year'],
                    'Author': author['name'],
                    'PublicationType': result['publicationTypes'], 
                    'CitationCount': result['citationCount'],
                    'FieldsOfStudy': result['fieldsOfStudy'],
                    'FieldsOfStudy_s2': result['s2FieldsOfStudy'],
                    'Journal': result['journal'],
                    'URL': result['url']})
    print(f'Since 2012 until {year} there are {len(data_authors)} results')
    

100


### Import data for Journal name and fileds of research

In [134]:
data_journals= []
df_journals = DataFrame(columns = ['Title', 
    'Year', 
    'FieldsOfStudy', 
    'FieldsOfStudy_s2', 
    'Journal'])

sch = SemanticScholar()
for year in range(2012, 2024):
    results = sch.search_paper('+"gender bias"', 
        fields=['title','year', 'fieldsOfStudy', 's2FieldsOfStudy', 'journal'],
        year=year)
    print(len(results))
    print(results[0])
    """
    for result in results:
        data_journals.append({'Title': result['title'], 
            'Year': result['year'],
            'FieldsOfStudy': result['fieldsOfStudy'],
            'FieldsOfStudy_s2': result['s2FieldsOfStudy'],
            'Journal': result['journal']})
    print(f'Since 2012 until {year} there are {len(data_journals)} results')
    """
    

In [3]:
dict_TitleJournal = {}
for data in data_journals:
    #print(data['Journal'].keys())
    try:
        dict_TitleJournal[data['Title']] = [data['Year'], data['Journal']['name'], data['FieldsOfStudy'], data['FieldsOfStudy_s2']]
    except:
        pass

In [35]:
dict_TitleField = {}
for data in data_journals:
    #print(data['Journal'].keys())
    try:
        dict_TitleField[data['Title']] = data['FieldsOfStudy']
    except:
        pass

In [4]:
print(data_journals[13])

{'Title': "Mechanisms of women's own-gender bias and sex differences in memory for faces", 'Year': 2012, 'FieldsOfStudy': ['Psychology'], 'FieldsOfStudy_s2': [{'category': 'Psychology', 'source': 'external'}, {'category': 'Psychology', 'source': 's2-fos-model'}], 'Journal': {'name': '', 'volume': ''}}


In [5]:
print(len(dict_TitleJournal))
print(dict_TitleJournal["Mechanisms of women's own-gender bias and sex differences in memory for faces"])

102391
[2012, '', ['Psychology'], [{'category': 'Psychology', 'source': 'external'}, {'category': 'Psychology', 'source': 's2-fos-model'}]]


In [34]:
i = 0

with open('data/dict_TitleJournalName.csv', 'w') as f:
    for key in dict_TitleJournal.keys():
        try:
            f.write("%s,%s\n"%(key,dict_TitleField[key]))
        except:
            i += 1

print(i)


3359


write to csv

In [18]:
# open the file in the write mode
with open('data/semanticScholar_try_+gender_bias.csv', 'w', encoding='UTF8') as f:
    # create the csv writer
    writer = csv.writer(f)
    # write a row to the csv file
    writer.writerow(data_authors)

write to a DataFrame

In [5]:
df_authors = DataFrame(data_authors, columns = ['Title', 'Year', 'Author', 'PublicationType', 'CitationCount', 'URL'])
print(df_authors['Author'].nunique())

247671


check duplicates

In [88]:
df_au_dd = df_authors.drop_duplicates(subset=["Title", "Author"], keep="first")
print(len(df_authors), len(df_au_dd))

396553 357789


drop duplicates inplace

In [130]:
df_authors.drop_duplicates(subset=["Title", "Author"], keep="first", inplace=True, ignore_index=True)

cast Year as int

In [92]:
df_authors.Year.astype('Int64')

0         2012
1         2012
2         2012
3         2012
4         2012
          ... 
396548    2023
396549    2023
396550    2023
396551    2023
396552    2023
Name: Year, Length: 357789, dtype: Int64

Show main info about the Dataframe and number of unique names

In [93]:
print(df_authors.info())
print(df_authors.isnull().sum())
df_authors.Author.nunique()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 357789 entries, 0 to 396552
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Title            357789 non-null  object
 1   Year             357789 non-null  int64 
 2   Author           357789 non-null  object
 3   PublicationType  229206 non-null  object
 4   CitationCount    357789 non-null  int64 
 5   URL              357789 non-null  object
dtypes: int64(2), object(4)
memory usage: 19.1+ MB
None
Title                   0
Year                    0
Author                  0
PublicationType    128583
CitationCount           0
URL                     0
dtype: int64


247671

In [38]:
df_authors.to_csv('data/authors_2012-2023.csv')

### Sharpen the results

In [4]:
df_authors = read_csv('data/authors_2012-2023.csv')

In [9]:
print('3 patterns', len(df_authors[df_authors.Title.str.contains('[Gg]ender(.*)[Bb]ias|[Gg]ender(.*)[Gg]ap|[Gg]ender(.*)[iI]nequalit')]))
print('Gender bias', len(df_authors[df_authors.Title.str.contains('[Gg]ender(.*)[Bb]ias')]))
print('Gender Gap', len(df_authors[df_authors.Title.str.contains('[Gg]ender(.*)[Gg]ap')]))
print('Gender Inequality', len(df_authors[df_authors.Title.str.contains('[Gg]ender(.*)[iI]nequalit')]))
print('Gender difference', len(df_authors[df_authors.Title.str.contains('[Gg]ender(.*)[Dd]ifference')]))

C:\Users\afedorova\AppData\Local\Temp\ipykernel_12032\1389154377.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  print('3 patterns', len(df_authors[df_authors.Title.str.contains('[Gg]ender(.*)[Bb]ias|[Gg]ender(.*)[Gg]ap|[Gg]ender(.*)[iI]nequalit')]))


3 patterns 16353


C:\Users\afedorova\AppData\Local\Temp\ipykernel_12032\1389154377.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  print('Gender bias', len(df_authors[df_authors.Title.str.contains('[Gg]ender(.*)[Bb]ias')]))


Gender bias 8324


C:\Users\afedorova\AppData\Local\Temp\ipykernel_12032\1389154377.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  print('Gender Gap', len(df_authors[df_authors.Title.str.contains('[Gg]ender(.*)[Gg]ap')]))


Gender Gap 5327


C:\Users\afedorova\AppData\Local\Temp\ipykernel_12032\1389154377.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  print('Gender Inequality', len(df_authors[df_authors.Title.str.contains('[Gg]ender(.*)[iI]nequalit')]))


Gender Inequality 2881


C:\Users\afedorova\AppData\Local\Temp\ipykernel_12032\1389154377.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  print('Gender difference', len(df_authors[df_authors.Title.str.contains('[Gg]ender(.*)[Dd]ifference')]))


Gender difference 30100


In [377]:
df_authors_re = df_authors[df_authors.Title.str.contains('[Gg]ender(.*)[Bb]ias|[Gg]ender(.*)[Gg]ap|[Gg]ender(.*)[dD]ifference')]
df_authors_re = df_authors_re.reset_index(drop=True)
#df_authors_re.to_csv('data/authors_2012-2023_regex.csv')

C:\Users\afedorova\AppData\Local\Temp\ipykernel_1824\2789606413.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_authors_re = df_authors[df_authors.Title.str.contains('[Gg]ender(.*)[Bb]ias|[Gg]ender(.*)[Gg]ap|[Gg]ender(.*)[dD]ifference')]


In [378]:
print('number of row in the initial DF: ', len(df_authors), '\n', 'number of row in the DF after regex: ', len(df_authors_re), sep='')
print('number of unique articles before search precision:', df_authors['Title'].nunique())
print('number of unique article titles after search precision:',df_authors_re['Title'].nunique()) #.to_csv('data/regex_search.csv')
print('number of unique author names after search precision:',df_authors_re['Author'].nunique()) #.to_csv('data/regex_search.csv')

number of row in the initial DF: 357789
number of row in the DF after regex: 39287
number of unique articles before search precision: 99189
number of unique article titles after search precision: 10551
number of unique author names after search precision: 33793


### Data cleaning

#### Clean chinese characters

Search for any Chinese characters in Author names, as this is the crucial informaiton for our data.

In [320]:
df_authors_re[['Title', 'Author']][df_authors_re['Author'].str.contains(r'[\u4e00-\u9FFF]', regex=True)]

,Title,Author
4089,Gender Roles: Biases or Differences?,魏静
4179,從本土企業董事性別比例平衡與績效的關係看性別平等意涵; Corporate Board Ge...,王孟洵
7925,Gender Bias and Resistance in Edna O'Brien's C...,張崇旂
7996,Gender Bias Against Foreign Wives : Analysis o...,加藤 直子
9900,Lower Melatonin Secretion in Older Females: Ge...,Kenji 賢史 Obayashi 大林
9901,Lower Melatonin Secretion in Older Females: Ge...,Keigo 圭吾 Saeki 佐伯
9902,Lower Melatonin Secretion in Older Females: Ge...,Nobuhiro 庸浩 Tone 刀根
9903,Lower Melatonin Secretion in Older Females: Ge...,Junko 淳子 Iwamoto 岩本
9904,Lower Melatonin Secretion in Older Females: Ge...,Kimie 季美恵 Miyata 宮田
9905,Lower Melatonin Secretion in Older Females: Ge...,Yoshito 義人 Ikada 筏


Often if the author name has Chinese characters, also title would have them, but they are always accompanied with an English title, so we can remove Chinese characters and be sure, that the title will not ne NaN.

In [379]:
df_authors_re['Title'] = df_authors_re['Title'].str.replace(r'[\u4e00-\u9FFF]','', regex=True)

We will partially drop rows, where the author column only contains Chinese characters, if in this case the preliminary analysis has shown, that's it'snot possible to find out the full name of the author. For some titles this information was found and will be udated.

On the other hand we will keep the rows with mixed Latin and Chinese characters, as they provide the name information and just remove Chinese characters.

In [380]:
titles_drop = ['Deuce or advantage? : examining gender bias in online coverage of professional tennis through a mixed methodology',
    'Gender Roles: Biases or Differences?',
    'Gender Differences in Conversation',
    'Analysis of Gender Differences in Color Reaction in TV Show You Are the One',
    'Gender Bias and Stereotypes in English Example Sentences—Taking Linguistic Academic Journals as Example']
title_name_drop = ['; Corporate Board Gender Diversity and Firm Financial Performance：The Institutional Investor Bias in Taiwan',
    '王孟洵']

print(len(df_authors_re))

i_toDrop = df_authors_re[(df_authors_re.Title.isin(titles_drop)) |\
    ((df_authors_re.Title == title_name_drop[0]) & \
    (df_authors_re.Author == title_name_drop[1]))].index
df_authors_re = df_authors_re.drop(i_toDrop)

print(len(df_authors_re))

39287
39280


In [381]:
df_authors_re['Author'][df_authors_re['Author'].str.contains(r'[\u4e00-\u9FFF]', regex=True) & df_authors_re['Author'].str.contains(r'[a-zA-Z]', regex=True)] \
    = df_authors_re['Author'].str.replace(r'[\u4e00-\u9FFF]','', regex=True)

C:\Users\afedorova\AppData\Local\Temp\ipykernel_1824\4128585737.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_authors_re['Author'][df_authors_re['Author'].str.contains(r'[\u4e00-\u9FFF]', regex=True) & df_authors_re['Author'].str.contains(r'[a-zA-Z]', regex=True)] \


In [382]:
replace_ChinCharNames = {'張崇旂': 'Tsung-Chi Chang', #'Gender bias and resistance in Edna O’Brien’s “country girls' trilogy”' 
    '加藤 直子': 'Naoko Kato', #'Gender Bias Against Foreign Wives : Analysis of International Divorce Cases in Japan' 
    '三好 葉月': 'Hazuki Moyoshi', #'Reducing unconscious gender bias through workshop with co-speculative design'
    '加藤 朗': 'Akira Kato'} #'Reducing unconscious gender bias through workshop with co-speculative design'

df_authors_re['Author'] = df_authors_re['Author'].replace(replace_ChinCharNames, regex=True)

In [383]:
df_authors_re[['Title','Author']][df_authors_re['Author'].str.contains(r'[\u4e00-\u9FFF]', regex=True)]

,Title,Author


#### Search for Hangul (Korean characters)

In [327]:
df_authors_re[['Title','Author']][df_authors_re['Author'].str.contains(r'[\u3131-\u3163\uac00-\ud7a3]', regex=True)]

,Title,Author
154,Gender Bias in Schooling,김기승
3518,한국가족 내 부부간 성관계에 투영된 젠더 격차(Gender Gap),함인희
7894,Gender Differences in Oral Health Literacy rel...,이규은
7895,Gender Differences in Oral Health Literacy rel...,염영희
7896,Gender Differences in Oral Health Literacy rel...,김상숙
7897,Gender Differences in Oral Health Literacy rel...,한정희
7995,Cultural Linguistic Study on the Chinese Gende...,김현태
8730,A Study on Factors Influencing Elders' Suicida...,구춘영
8731,A Study on Factors Influencing Elders' Suicida...,김정순
8732,A Study on Factors Influencing Elders' Suicida...,유정옥


In [384]:
replace_KorCharNames = {'김기승': 'Seung-Gi Kim',
    'M. Milligan': 'Michael A. Milligan',
    '이규은': 'Kyu Eun lee', #Gender Differences in Oral Health Literacy
    '염영희': 'Young-Hee Yom', #Gender Differences in Oral Health Literacy
    '김상숙': 'Sang Suk Kim', #Gender Differences in Oral Health Literacy
    '한정희': 'Jung Hee Han', #Gender Differences in Oral Health Literacy
    '김현태': 'Kim Hyun Tae', #'Korean' 'Male' #'Cultural Linguistic Study on the Chinese Gender-biased Terms against Women'
    '구춘영': 'Chun Young Koo', #A Study on Factors Influencing Elders' Suicidal Ideation: Focused on Comparison of Gender Differences
    '김정순': 'Jung Soon Kim', #A Study on Factors Influencing Elders' Suicidal Ideation: Focused on Comparison of Gender Differences
    '유정옥': 'Jungok Yu', #A Study on Factors Influencing Elders' Suicidal Ideation: Focused on Comparison of Gender Differences
    '김세현': 'Sehyoun Kim'} #An Audience-response Questionnaire Survey on Gender-biased Language in Korean Subtitling}

df_authors_re['Author'] = df_authors_re['Author'].replace(replace_KorCharNames, regex=True)
i_toDrop = df_authors_re[df_authors_re.Title == '한국가족 내 부부간 성관계에 투영된 젠더 격차(Gender Gap)'].index
df_authors_re = df_authors_re.drop(i_toDrop)

len(df_authors_re)


39279

#### Check rows with names, that only contain one word

In [392]:
df_authors_re[['Title', 'Author']][~df_authors_re.Author.str.contains('\s')] #.to_csv('data/onewordnames.csv')
    #'^[\S]\.[\S].')] 
    # #& ~df_authors_re.Author.str.contains('^[a-zA-Z]\.\s[\S].*')]


,Title,Author


In [370]:
print(df_authors_re['Author'][df_authors_re.Author.isin(names_short)])

Series([], Name: Author, dtype: object)


In [393]:
names_update = {'K: Hughes': 'Karen D. Hughes',
'S. Albanesi': 'Stefania Albanesi',
'Nyfed': 'Aysegül Sahin',
'Bankole': 'Bankole Fasanya',
'Maranda': 'Maranda Mcbride',
'Pope-Ford': 'Regina Pope-Ford',
'Ntuen': 'Celestine Ntuen',
'E. Alpha': 'Ayande Alpha',
'Quinetta': 'Quinetta M. Roberson',
'Hrebickova': 'Hana Hrebickova',
#'M.': 'Gregory S. Parks',
'E. Oyeniyi': 'Akande Samson Oyeniyi',
'ZhangXiaofei': 'Zhang Xiaofei',
'GuoXitong': 'Guo Xitong',
'LaiKee-hung': 'Lai Kee-hung',
'GuoFeng': 'Guo Feng',
'LiChenlei': 'Li Chenlei',
'CohenAaron': 'Aaron Cohen',
'Ein-MorEliana': 'Eliana Einmor',
'StessmanJochanan': 'Jochanan Stessman',
'B. Rustin': 'Brett Rustin Nielsen',
'R. Foels': 'Rob Foels',
'I. GuerdjikovaAnna': 'Anna I. Guerdjikova',
'J. BlomThomas': 'Thomas J. Blom',
'MoriNicole': 'Nicole Mori',
'CasutoLeah': 'Leah Casuto',
'KeckPaul E.Jr.': 'Paul E.Jr. Keck',
'L. McElroySusan': 'Susan McElroy',
'TerranovaVictoria': 'Victoria Terranova',
'M. VandiverDonna': 'Donna Vandiver',
'GuoXitong': 'Guo Xitong',
'HanXiaocui': 'Han Xiaocui',
'ZhangXiaofei': 'Zhang Xiaofei',
'DangYuanyuan': 'Dang Yuanyuan',
'ChenChun': 'Chen Chun',
'Corrine': 'Corinne Wilsey',
'Hazel-FernandezLeslie': 'Leslie Hazel-Fernandez',
'NeroDamion': 'Damion Nero',
'MoretzChad': 'Chad Moretz',
'SlabaughLane': 'Lane Slabaugh',
'MeahYunus': 'Yunus Meah',
'BaltzJean': 'Jean Baltz',
'CostantinoMary': 'Mary Costantino',
'BouchardJonathan': 'Jonathan Bouchard',
'Omiya': 'Kazuto Omiya',
'S. Gilli': 'Sofia Gilli',
'TempletonKim': 'Kim Templeton',
'S. Javed': 'Saira Javed',
'Umm-e-Siddiqa': 'Umme Siddiqa',
'IramZehraBokharey': 'Zehra Iram Bokharey',
'J. Turabián': 'Jose Turabián',
'ra Moreno-Ruiz': 'Sandra Moreno-Ruiz',
'R. Cucho-Jove': 'Raul Cucho-Jove',
'ro Villarin-Castro': 'Alejandro Villarin-Castro',
'P. Bhattacharya': 'Prabir Bhattacharya',
'Amsterdam': 'Marja J Aartsen',
'S. Yamtinah': 'Sri Yamtinah',
'M. Masykuri': 'Mohammad Masykuri',
'A. Ashadi': 'Ashadi Ashadi',
'A. Shidiq': 'Ari Syahidul Shidiq',
'CarapinhaRené': 'René Carapinha',
'E. Choo': 'Esther Choo',
'R. DeMayo': 'Robert DeMayo',
'Warnakula':'Upuli Warnakula',
"Stephen D. O'Connell": 'Stephen D. OConnell',
"O'Connell": 'Stephen D. OConnell',
'Baloglu': 'Mustafa Baloğlu',
'AuerMichael': 'Michael Auer',
'FrauscherBirgit': 'Birgit Frauscher',
'HochleitnerMargarethe': 'Margarethe Hochleitner',
'HöglBirgit': 'Birgit Högl',
'Mzamu-Kampanje': 'Jessica Mzamu-Kampanje',
'SudomKerry': 'Kerry Sudom',
'GuérinEva': 'Eva Guérin',
'E. C. LeeJennifer': 'Jennifer E.C. Lee',
'S. Ananiadou': 'Sophia Ananiadou',
'Odbal': 'Odbal Odbal',
'Y. Hirota': 'Yusuke Hirota',
'y-hirota': 'Yuta Nakashima', 
'O. Steinlein': 'Ortrud Steinlein',
'M. Reithmair': 'Marlene Reithmair',
'ya Syunyaeva': 'Zulfiya Syunyaeva',
'E. Sattler': 'Elke C Sattler',
'Anthonius':'Anthonius de Boer',
'Denier': 'Nicole Denier',
'Gag': 'Gagandeep Singh',
'M. Abiodun':'Abiodun M. Gesinde',
'Js Oluwafunto':'Oluwafunto J. Sanu'}

df_authors_re['Author'] = df_authors_re['Author'].replace(names_update)


In [390]:
add_lines = {'Author': ['Mike Martin',
        'Daniel Zimprich', 
        'Noa Garcia', 
        'Gregory S. Parks', 
        'Rodney M. Miller', 
        'Ashadi Ashadi', 
        'Cecilia O. Alm',
        'Reeta Karra',
        'Jennifer Lyke',
        'Yong Li',
        'Jana Vasickova',
        'Dorota Groffik',
        'Sharon Luise Njet-Moi Tjin A Tsoi',
        'Andries Koster'
        ],
    'Title' : ['Gender differences in level and change in cognitive functioning : Results from the Longitudinal Aging Study',
        'Gender differences in level and change in cognitive functioning : Results from the Longitudinal Aging Study',
        'Gender and Racial Bias in VisualQuestion Answering Datasets',
        'Eighteen Million Cracks: Genders Role in the 2008 Presidential Campaign',
        'If the Shoe Fits: A Historical Exploration of Gender Bias in the U.S. Sneaker Industry',
        "Gender differences in students’ attitudes toward science: An analysis of students’ science process skill using testlet instrument",
        'Analyzing Gender Bias in Student Evaluations Acknowledgments',
        'Gender Bias in Availability of School Education in Villages - A Study of Kalisindh Thermal Power Project',
        'Gender Differences in Perceived Illness, Stress, and Coping in Undergraduates',
        'Racial/ethnic and gender differences in severity of diabetes-related complications, health care resource use, and costs in a Medicare population.',
        'Gender, Age and Body Mass Differences Influencing the Motivation for Physical Activity among Polish Youths',
        'Gender, Age and Body Mass Differences Influencing the Motivation for Physical Activity among Polish Youths',
        'Gender Differences in Motivation and Learning Preferences of Pharmacists in Lifelong Learning',
        'Gender Differences in Motivation and Learning Preferences of Pharmacists in Lifelong Learning'
]}

df_authors_re = df_authors_re.append(DataFrame(add_lines,columns=['Author', 'Title'])).reset_index(drop=True)
df_authors_re.tail()

C:\Users\afedorova\AppData\Local\Temp\ipykernel_1824\1923523578.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_authors_re = df_authors_re.append(DataFrame(add_lines,columns=['Author', 'Title'])).reset_index(drop=True)


,Title,Year,Author,PublicationType,CitationCount,URL
39211,Racial/ethnic and gender differences in severi...,NaN,Yong Li,NaN,NaN,NaN
39212,"Gender, Age and Body Mass Differences Influenc...",NaN,Jana Vasickova,NaN,NaN,NaN
39213,"Gender, Age and Body Mass Differences Influenc...",NaN,Dorota Groffik,NaN,NaN,NaN
39214,Gender Differences in Motivation and Learning ...,NaN,Sharon Luise Njet-Moi Tjin A Tsoi,NaN,NaN,NaN
39215,Gender Differences in Motivation and Learning ...,NaN,Andries Koster,NaN,NaN,NaN


In [722]:
titles_drop2 = ['Gender-biased Diagnosing', 
    'Opiorphin Secretion Pattern in Healthy Volunteers: Gender Difference andOrgan Specificity',
    "Gender Bias in Women's Health, Obstetric, and Neonatal Nursing.",
    'Featured Article: Gender Bias in Pediatric Pain Assessment',
    'Gender Difference in the Effects of COVID-19 Pandemic on Mechanical Reperfusion and 30-Day Mortality for STEMI: Results of the ISACS-STEMI COVID-19 Registry']

i_toDrop = df_authors_re[df_authors_re['Title'].isin(titles_drop2)].index
df_authors_re = df_authors_re.drop(i_toDrop)

In [389]:
print(len(df_authors_re))
i_noSpace = df_authors_re[~df_authors_re.Author.str.contains('\s')].index
df_authors_re = df_authors_re.drop(i_noSpace)
print(len(df_authors_re))

39268
39202


#### Other strange cases

Rename some titles, that don't correspond fully to original paper title.

In [394]:
# 'https://www.semanticscholar.org/paper/b5612cc672817a7d65faf8c273550858b63790a2'
rename_title = {'Re; Pandit SR, Venugopal P, Keshavamurthy R, Chawla A. Challenges and gender-based differences for women in the Indian urological workforce: Results of a survey. Indian J Urol 2022;38:282-6':\
        'Challenges and gender-based differences for women in the Indian urological workforce: Results of a survey',
    'SUCCESSFUL TITLE VII CLAIM FOR IMPLICIT GENDER BIAS ? A . Implicit Bias : A Definition B . Implicit Gender Bias , Title VII , and the Clinton Campaign':\
        'Eighteen Million Cracks: Genders Role in the 2008 Presidential Campaign' }

df_authors_re['Title'] = df_authors_re.Title.replace(rename_title)

name_toDrop = df_authors_re[df_authors_re.Author == 'for the Colon Cancer Family Registry'].index
df_authors_re = df_authors_re.drop(name_toDrop)

Check the rename results

In [396]:
df_authors_re['Title'][df_authors_re['Title'].isin(rename_title.values())]

Series([], Name: Title, dtype: object)

In [397]:
df_authors_re.to_csv('data/authors_2012-2023_regex.csv')

### Find rows with full names

In [17]:
df_authors_re = read_csv('data/df_authors_full.csv')
df_authors_reduced = df_authors_re[df_authors_re.Title.str.contains('[Gg]ender(.*)[Bb]ias|[Gg]ender(.*)[Gg]ap|[Gg]ender(.*)[iI]nequalit')] #.to_csv('data/df_authors_re-genderDifference.csv')

C:\Users\afedorova\AppData\Local\Temp\ipykernel_12032\5840737.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_authors_reduced = df_authors_re[df_authors_re.Title.str.contains('[Gg]ender(.*)[Bb]ias|[Gg]ender(.*)[Gg]ap|[Gg]ender(.*)[iI]nequalit')] #.to_csv('data/df_authors_re-genderDifference.csv')


create two DF, with full names and with short names

In [486]:
df_authors_full = df_authors_re[~df_authors_re.Author.str.contains('^[\S]\.\s[\S].*|^[\S]\.\s[\S]\.|^[\S]\.[\S].')].reset_index(drop=True)
    #~df_authors_re.Author.str.contains('^[\S]\.\s[\S]\.*]')]
df_authors_short = df_authors_re[df_authors_re.Author.str.contains('^[\S]\.\s[\S].*|^[\S]\.\s[\S]\.|^[\S]\.[\S].')].reset_index(drop=True)
    #df_authors_re.Author.str.contains('^[\S]\.\s[\S]\.')]
print('Length of DF with full names is ', len(df_authors_full), '\n', 'Length of DF with short names is ', len(df_authors_short), sep='')

Length of DF with full names is 15516
Length of DF with short names is 23700


In [170]:
df_authors_full['Author'][df_authors_full.FirstName.isna()] #[df_authors_full.FirstName.str.contains('^[\S]\.\s[\S].*|^[\S]\.\s[\S]\.|^[\S]\.[\S].')]

15510        R. Dietz
15511      R. Pearson
15512        M. Semak
15513       C. Willis
15527       A. Little
             ...     
34325     P. Zeppegno
34326    S. Galderisi
34327          M. Maj
34328       S. Mandal
34329        P. Nayak
Name: Author, Length: 8805, dtype: object

check if there are any authors in both df due to a mistake in regex

In [399]:
df_authors_full[df_authors_full['Author'].isin(df_authors_short['Author'])] #[df_authors.Author.str.contains('^[A-Za-z]{1}\.\s[A-Za-z-]{2,50}')]

,Title,Year,Author,PublicationType,CitationCount,URL


check some problematic cases

In [400]:
print(df_authors_short['Author'][df_authors_short['Author']=='G. Ríos-Muñoz'],
    df_authors_short['Author'][df_authors_short['Author']=='A. van Mark'],
    df_authors_short['Author'][df_authors_short['Author']=='Á. M. Fidalgo'],
    df_authors_short['Author'][df_authors_short['Author']=='C. Çöteli'], 
    df_authors_short['Author'][df_authors_short['Author']=='K. Židová'], 
    df_authors_short['Author'][df_authors_short['Author']=='Á. Szabó-Morvai'], sep='\n')

37272    G. Ríos-Muñoz
Name: Author, dtype: object
34619    A. van Mark
Name: Author, dtype: object
20307    Á. M. Fidalgo
Name: Author, dtype: object
27359    C. Çöteli
Name: Author, dtype: object
3357    K. Židová
Name: Author, dtype: object
38231    Á. Szabó-Morvai
Name: Author, dtype: object


Print the info about resulting dataframes
- 10523 publications have 33714 full names
- 3326 publications have both forms of author names
- 8101 author names have to be corrected
- 7197 publications have 0 authors with only short forms 

In [89]:
#print(f'In the final search there are {df_authors_re.Title.nunique()} publications from {df_authors_re.Author.nunique()} authors')
print(f'{df_authors_full.Title.nunique()} publications have {df_authors_full.Author.nunique()} full names')
print(f'{df_authors_full[df_authors_full.Title.isin(df_authors_short.Title.unique())].Title.nunique()} publications have both forms of author names')
print(f'{df_authors_short[df_authors_short.Title.isin(df_authors_full.Title.unique())].Author.nunique()} author names have to be corrected')
print(f'{df_authors_full[~df_authors_full.Title.isin(df_authors_short.Title.unique())].Title.nunique()} publications \
have {df_authors_short[~df_authors_short.Title.isin(df_authors_full.Title.unique())].Author.nunique()} authors with only short forms ')

10523 publications have 33714 full names
3326 publications have both forms of author names
8101 author names have to be corrected
7197 publications have 0 authors with only short forms 


In [488]:
df_authors_full.Year.value_counts(sort=False)

2012.0    1190
2013.0    1106
2014.0    1183
2015.0    1283
2016.0    1377
2017.0    1510
2018.0    1506
2019.0    1580
2020.0     633
2021.0    1889
2022.0    1817
2023.0     428
Name: Year, dtype: int64

In [95]:
def name_separator(fullName):
    """Splits a full name into first name, last name and middle name
    
    :param: fullName string with the full name
    :return: firstName string first part of the full name string
    :return: lastName string last part of the full name string
    :return: middleName string if splitted full name contains more than 2 parts, everything in the middle
    """
    name = fullName.split()
    if len(name) > 2:
        firstName = name[0]
        lastName = name[-1]
        middleName = ' '.join(name[1:-1])
        return firstName, lastName, middleName
    elif len(name) == 2:
        firstName = name[0]
        lastName = name[1]
        return firstName, lastName, None

In [ ]:
df_authors_full[['FirstName', 'LastName', 'MiddleName']] = \
    df_authors_full.apply(lambda x: name_separator(x.Author), axis=1, result_type="expand")

In [ ]:
# Alternative: https://stackoverflow.com/questions/38437847/pandas-split-name-column-into-first-and-last-name-if-contains-one-space
# name_df[['first_name','last_name']] = name_df['name'].loc[name_df['name'].str.split().str.len() == 2].str.split(expand=True)

In [490]:
df_authors_full[['Author','FirstName', 'LastName', 'MiddleName']][~df_authors_full['MiddleName'].isna()]

,Author,FirstName,LastName,MiddleName
0,Laura V. Zimmermann,Laura,Zimmermann,V.
1,Seyyed Ahmad Mousavi,Seyyed,Mousavi,Ahmad
4,Margaret S. Stockdale,Margaret,Stockdale,S.
6,Seyyed Ahmad Mousavi,Seyyed,Mousavi,Ahmad
7,Judith Sama Yeba,Judith,Yeba,Sama
...,...,...,...,...
15487,Anna M. Stadelman,Anna,Stadelman,M.
15505,Gregory S. Parks,Gregory,Parks,S.
15506,Rodney M. Miller,Rodney,Miller,M.
15508,Cecilia O. Alm,Cecilia,Alm,O.


### Group by title to ind out, if all authors have names in the same format

In [75]:
author_gr = df_authors_re.groupby('Title')['Author'].apply(lambda x: "{%s}" % ', '.join(x))
type(author_gr)

pandas.core.series.Series

In [ ]:
#author_gr.str.contains('[A-Za-z]{1}\.\s[A-Za-z]{2,50}')

To update manually:

In [117]:
df_authors_short[df_authors_short.Title.isin(df_authors_full.Title.unique())].to_csv('data/short_names_to_fill.csv')

In [24]:
df_authors_re[df_authors_re.Title == 'Reducing unconscious gender bias through workshop with co-speculative design']

NameError: name 'df_authors_re' is not defined

In [ ]:
7926 'Gender bias and resistance in Edna O’Brien’s “country girls' trilogy”' 'Tsung-Chi Chang'
'Gender Bias Against Foreign Wives : Analysis of International Divorce Cases in Japan' 'Naoko Kato'
'Cultural Linguistic Study on the Chinese Gender-biased Terms against Women' 'Kim Hyun Tae' 'Korean' 'Male'
'Reducing unconscious gender bias through workshop with co-speculative design' 'Hazuki Moyoshi'
'Reducing unconscious gender bias through workshop with co-speculative design' 'Akira Kato'

'Odbal' 'female'

From https://aclanthology.org/2022.ccl-1.46.pdf:
36393-36398


# Is it possible to extract more names from initials?

In [491]:
df_authors_short[['FirstName', 'LastName', 'MiddleName']] = df_authors_short.apply(lambda x: name_separator(x.Author), axis=1, result_type="expand")

In [432]:
df_authors_short[df_authors_short['LastName'].isna()]

,Title,Year,Author,PublicationType,CitationCount,URL,FirstName,LastName,MiddleName


There are some erroneous firstNames, that must be filtered out during search: 
- one letter initials with a dot (eg. 'M.'),
- initials with two letters and dash or dot in between (eg. 'M. C.' or 'M.-C.')
- a title, that was collected as Name (eg. 'Dr.', 'Mr.)

They need to be cleared, but without erasing some names, that have dots at the end (eg. 'Jennifer.')

In [69]:
def update_names_from_RG(paper_title, surname,webdriver='dr', name=None):
    time.sleep(random.randint(1,10))
    query = paper_title.replace(' ', '+') + '+' + surname
    url = f"https://www.researchgate.net/search.Search.html?query={query}&type=publication"
    paper_title = re.sub('[^0-9a-zA-Z ]+', '', paper_title).lower()
    #print('Search for:', paper_title)
    dr.get(url)
    soup = BeautifulSoup(dr.page_source,"lxml")

    results = soup.find_all('div', class_="nova-legacy-v-publication-item__stack nova-legacy-v-publication-item__stack--gutter-m")
        
    for result in results:
        result_title = result.find_all("a", class_="nova-legacy-e-link nova-legacy-e-link--color-inherit nova-legacy-e-link--theme-bare")
        for result_t in result_title:
            result_t = re.sub('[^0-9a-zA-Z ]+', '', result_t.text).lower()
            #print(result_t)
            #print('Found:', re.sub('[^0-9a-zA-Z ]+', '', result_t),         
            if result_t == paper_title:
                results_name = result.find_all('a', class_='nova-legacy-v-person-inline-item')
                for name in results_name:
                    full_name = name.text.split(' ')
                    #print(surname, full_name[-1])
                    if surname.lower() == full_name[-1].lower():
                        if not full_name[0].replace('.', '').isupper():
                            if (full_name[0] not in ['Mr.', 'Dr.', 'Miss., Md.']):
                                first_name = full_name[0]
                                #print(first_name)
                                return first_name

Open Chrome Tab for search

In [71]:
dr = webdriver.Chrome()

Perform search in batches of 50 rows, appending the search result in FirstName column.

Every 500 rows the results are saved as csv.

In [54]:
for i in range(0,11739,50):
    df_authors_short_batch = df_to_expand.iloc[i:i+50]
    df_authors_short_batch['FirstName'] = df_authors_short_batch.apply(lambda x: update_names_from_RG(x.Title, x.LastName, dr), axis=1)
    df_authors_full2 = df_authors_full2.append(df_authors_short_batch, ignore_index=True)
    print(f'{i+50} done')
    if (i + 50) % 500 == 0:
        #dr = webdriver.Chrome()
        df_authors_full2[~df_authors_full.FirstName.isnull()].to_csv('data/authors_full_names_extended.csv')

Results of update and the rate of success (number of update names in proportion to all added names)

In [204]:
print('Added new full names: ',len(df_authors_full[~df_authors_full.FirstName.isnull()]) - 15500, '\n',
    'Could not update names: ', len(df_authors_full[df_authors_full.FirstName.isnull()]), '\n',
    'success rate = ', ((len(df_authors_full[~df_authors_full.FirstName.isnull()]))-15500)/23700, sep='')

Added new full names: 17477
Could not update names: 8805
success rate = 0.7374261603375527


Added new full names: 17477

Could not update names: 8805

success rate = 0.7374261603375527

In [87]:
df_authors_short = df_authors_full[df_authors_full.FirstName.isnull()]

In [164]:
df_authors_full.to_csv('data/df_authors_full.csv')

# Analysis of the short and full names ratio after search in ResearchGate

32855 8785 before manual update

34059 7581 after manual update

In [79]:
df_full = df_authors_full[df_authors_full.FirstName.notna()]
df_short = df_authors_full[df_authors_full.FirstName.isna()]
print(len(df_full), len(df_short))

34059 7581


In [80]:
#print(f'In the final search there are {df_authors_re.Title.nunique()} publications from {df_authors_re.Author.nunique()} authors')
print(f'{df_full.Title.nunique()} publications have {df_full.Author.nunique()} full names')
print(f'{df_full[df_full.Title.isin(df_short.Title.unique())].Title.nunique()} publications have both forms of author names')
print(f'{df_short[df_short.Title.isin(df_full.Title.unique())].Author.nunique()} author names have to be corrected')
print(f'{df_short[~df_short.Title.isin(df_full.Title.unique())].Title.nunique()} publications \
have {df_short[~df_short.Title.isin(df_full.Title.unique())].Author.nunique()} authors with only short forms ')

10754 publications have 29584 full names
2237 publications have both forms of author names
5692 author names have to be corrected
716 publications have 1461 authors with only short forms 


before manual update:

10735 publications have 28733 full names

2577 publications have both forms of author names

6675 author names have to be corrected

735 publications have 1525 authors with only short forms

Create two dataFrames grouped by Title and the column "Count" shows, how many times the title appears in the initial DataFrame. It shows, how many authors were identified (for df_full) or could not be identified (df_short) for each title.

In [81]:
df_short_group = df_short.groupby(['Title'])['Title'].count().reset_index(
  name='Count').sort_values(['Count'], ascending=False)
df_full_group = df_full.groupby(['Title'])['Title'].count().reset_index(
  name='Count').sort_values(['Count'], ascending=False)

print_all(df_full_group)

                                                   Title  Count
4240   Gender Gap in Parental Leave Intentions: Evide...     80
2925   Gender Differences in 3-Month Outcomes of Eren...     36
795    Author Correction: Within-job gender pay inequ...     29
10532   Within-job gender pay inequality in 15 countries     29
6475   Gender inequality and not female mentors hinde...     28
3073   Gender Differences in Case Volume Among Ophtha...     28
813    Awareness of sex and gender dimensions among p...     26
203    A comparative biomarker study of 514 matched c...     24
6646   Gender- and Age-Specific Differences in the As...     22
4785   Gender bias in academia: A lifetime problem th...     22
3237   Gender Differences in Endovascular Revasculari...     21
2998   Gender Differences in Associations of Glutamat...     21
1103   Cisplatin-Induced Nephrotoxicity; Protective S...     21
2979   Gender Differences in Alzheimer Disease: Brain...     21
2803   Gender Difference in the Effects 

Have a look, what is a proportion of authors with short and full names for each publication, that needs full names. 

Suggestion: only update manually names, if there is 1 or 2 names missing for a publication. 

We can calculate proportion of short names compared to the number of authrs for each publication and only fill in the short names, if the proportion of missing names is below 60 % (?).

assign to the group with full names the counts of names in the short table

calculate the proportion of names with full names among all names in publication

In [91]:
df_full_group = df_full_group.assign(
    Count_short=lambda df: df_short_group['Count'][df["Title"].isin(df_short_group["Title"])])
df_full_group['Count_short'] = df_full_group['Count_short'].astype('Int64')
df_full_group['Proportion_names'] = df_full_group.apply(lambda x: (x.Count)/(x.Count + x.Count_short), axis=1)

#df_full_group['Proportion_names'] = df_full_group['Proportion_names'].astype('Int64')

Most popular titles are defined as publications with proportion of full names over 60 %

In [237]:
title_most_popular = [l for l in df_full_group['Title'][df_full_group.Proportion_names > 0.6]]

Do the same for short names

In [98]:
#df_short_group[df_short_group["Title"].isin(df_full_group["Title"])] = df_short_group.assign(
#    Count_full=lambda df: df_full_group['Count'])

df_short_group['Count_full'] = df_short_group['Title'].map(Series(df_full_group.Count.values,index=df_full_group.Title))
df_short_group['Count_full'] = df_short_group['Count_full'].astype('Int64')
df_short_group['Proportion_names'] = df_short_group.apply(lambda x: (x.Count)/(x.Count + x.Count_full), axis=1)
print_all(df_short_group[['Count','Count_full', 'Proportion_names']].sort_values('Proportion_names', ascending=False))

      Count  Count_full Proportion_names
1494     14           1         0.933333
31       85           9         0.904255
171       9           1              0.9
1491      9           1              0.9
1688     26           3         0.896552
1899      8           1         0.888889
1217     40           5         0.888889
1288      8           1         0.888889
3         8           1         0.888889
11        8           1         0.888889
2361      8           1         0.888889
657       8           1         0.888889
1645     15           2         0.882353
2637     59           8         0.880597
297       7           1            0.875
1362      7           1            0.875
2196      7           1            0.875
1888      7           1            0.875
466       7           1            0.875
180       7           1            0.875
1310      7           1            0.875
1503      7           1            0.875
1097     14           2            0.875
343       7     

In [ ]:
df_short[['Title', 'Author']][df_short.Title.isin(title_most_popular)].sort_values('Title')

Another way to define the names, for which the enhancement will be most productive, is to choose the names, that are most frequent in the short table.

In [108]:
df_short_name = df_short[df_short.Title.isin(df_full.Title)].groupby(['Author'])['Author'].count().reset_index(
  name='Count').sort_values(['Count'], ascending=False)#.to_csv('data/most_popular_short_names.csv')
print(df_short_name)

                   Author  Count
982       C. Moss‐Racusin     15
1197        D. Gómez-Baya      7
4365            R. Böheim      5
3020         L. Dell’Osso      5
4455             R. Jagsi      5
...                   ...    ...
1950  H. Barajas-Martinez      1
1949           H. Bagheri      1
1947              H. Baba      1
1946              H. Arai      1
5691     Ž. Petrulionienė      1

[5692 rows x 2 columns]


In [105]:
dict_most_popular_names = Series(df_short_name.Count.values,index=df_short_name.Author)

In [117]:
df_short['Count_names'] = df_short.Author.map(Series(df_short_name.Count.values,index=df_short_name.Author))
df_short['Count_names'] = df_short['Count_names'].astype('Int64')
df_short[['Title', 'Author', 'Count_names']][df_short.Count_names.notnull()].sort_values(['Count_names', 'Author'], ascending=False).to_csv('data/most_popular_short_names.csv')

C:\Users\afedorova\AppData\Local\Temp\ipykernel_4432\2002589272.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_short['Count_names'] = df_short.Author.map(Series(df_short_name.Count.values,index=df_short_name.Author))
C:\Users\afedorova\AppData\Local\Temp\ipykernel_4432\2002589272.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_short['Count_names'] = df_short['Count_names'].astype('Int64')


In [174]:
df_short[df_short.Title.isin(df_full.Title.unique())].to_csv('data/short_names_to_fill.csv')

### Add manually found full names

In [174]:
df_mostPopularTitles = read_csv('data/most_popular_titles_enriched.csv', index_col=False, sep = ';',encoding_errors='ignore')
index = df_mostPopularTitles[df_mostPopularTitles.FirstName == 'delete'].index
df_mostPopularTitles = df_mostPopularTitles.drop(index)
df_mostPopularTitles

,Title,Author,FirstName,Gender
0,(Measurement of Occupational Gender Bias in Ch...,M. Guo,Mengqing,Male
1,(Measurement of Occupational Gender Bias in Ch...,Y. Liu,Ying,Male
2,062 VGLL3 is a key regulator of gender biased ...,J. Gudjonsson,Johann,Male
3,062 VGLL3 is a key regulator of gender biased ...,J. M. Kahlenberg,Joanne,Female
4,062 VGLL3 is a key regulator of gender biased ...,P. Harms,Paul,Male
...,...,...,...,...
1180,Gender Differences in Health Care Workers' Ris...,F. Trudzinski,Franziska,Female
1181,Gender Differences in Health Care Workers' Ris...,M. Kreuter,Michael,Male
1182,Gender Differences in Health Care Workers' Ris...,F. Herth,Felix,Male
1183,Gender Differences in Health Care Workers' Ris...,J. Bullwinkel,Jrn,Male


Create a dictionary with full names as key and first names as values.

In [ ]:
dict_FirstName_enriched = Series(df_mostPopularTitles.FirstName.values,index=df_mostPopularTitles.Author).to_dict()
print(dict_FirstName_enriched)

Add first names from the dictionary to the dataset

Number of short names reduced from 8785 to 7581.

In [176]:
print(len(df_authors_full[df_authors_full.FirstName.isna()]))

df_authors_full['FirstName'][df_authors_full['Author'].isin(df_mostPopularTitles.Author)] = df_authors_full['Author'].map(dict_FirstName_enriched)

print(len(df_authors_full[df_authors_full.FirstName.isna()]))

7581
7581


C:\Users\afedorova\AppData\Local\Temp\ipykernel_6064\3297512722.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_authors_full['FirstName'][df_authors_full['Author'].isin(df_mostPopularTitles.Author)] = df_authors_full['Author'].map(dict_FirstName_enriched)


For most popular names the number of authors with short names changes: from 7555 to 7183

In [197]:
df_mostPopularNames = read_csv('data/most_popular_short_names_enriched.csv', index_col=False, sep = ';',encoding_errors='ignore')
index = df_mostPopularNames[df_mostPopularNames.FirstName == 'delete'].index
df_mostPopularNames = df_mostPopularNames.drop(index)
df_mostPopularNames = df_mostPopularNames.drop('Title', axis=1)
df_mostPopularNames = df_mostPopularNames[df_mostPopularNames.FirstName.notna()]
df_mostPopularNames = df_mostPopularNames.drop_duplicates()
df_mostPopularNames

dict_FirstName2_enriched = Series(df_mostPopularNames.FirstName.values,index=df_mostPopularNames.Author).to_dict()
#print(dict_FirstName2_enriched)
print(len(df_authors_full[df_authors_full.FirstName.isna()]))

df_authors_full['FirstName'][df_authors_full['Author'].isin(df_mostPopularNames.Author)] = \
    df_authors_full['Author'].map(dict_FirstName2_enriched)

print(len(df_authors_full[df_authors_full.FirstName.isna()]))

7555
7183


C:\Users\afedorova\AppData\Local\Temp\ipykernel_6064\4135028803.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_authors_full['FirstName'][df_authors_full['Author'].isin(df_mostPopularNames.Author)] = \


### Analysis of multiple rows with the same full name - extending of information, if in some publications the first name was omitted.

In [318]:
df_authors_with_names = df_authors_full[['Author','LastName','FirstName']][df_authors_full.FirstName.notnull()].copy()
df_authors_with_names = df_authors_with_names.drop_duplicates(['Author', 'FirstName'])
df_authors_no_names = df_authors_full[['Author','LastName','FirstName']][df_authors_full.FirstName.isnull()].copy()
df_authors_no_names = df_authors_no_names.drop_duplicates(['Author', 'LastName'])
df_authors_no_names = df_authors_no_names[df_authors_no_names.Author.isin(df_authors_with_names.Author)]


In [319]:
print(len(df_authors_with_names), len(df_authors_no_names))

30211 49


Only leave authors, whose full name appears in both dataframes, so it has records with both full and short names

In [320]:
df_authors_with_names = df_authors_with_names[df_authors_with_names.Author.isin(df_authors_no_names.Author)].append(df_authors_no_names)
len(df_authors_with_names)

C:\Users\afedorova\AppData\Local\Temp\ipykernel_6064\2145175219.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_authors_with_names = df_authors_with_names[df_authors_with_names.Author.isin(df_authors_no_names.Author)].append(df_authors_no_names)


98

Delete the authors, whose name appears more than 2 times.

In [321]:
index = df_authors_with_names.Author.value_counts().loc[lambda x: x>2]
df_authors_with_names = df_authors_with_names[~df_authors_with_names.Author.isin(index)]
len(df_authors_with_names)

98

Delete short names by calculating the length of names and dropping the 1 letter names.

In [309]:
#df_authors_with_names[df_authors_with_names.FirstName.notna()].FirstName.str.contains(r'^[A-Z]$')
df_authors_with_names.FirstName = df_authors_with_names.FirstName.str.rstrip('.')
#print_all(df_authors_with_names)
df_authors_with_names['NameLength'] = df_authors_with_names['FirstName'].str.len()
df_authors_with_names = df_authors_with_names.drop(df_authors_with_names[df_authors_with_names.NameLength == 1].index)
df_authors_with_names = df_authors_with_names.sort_values('Author')

Create a dictionary from full names and first names.

In [290]:
dict_names_doubles = Series(df_authors_with_names.FirstName[df_authors_with_names.FirstName.notna()].values,
                            index=df_authors_with_names.Author[df_authors_with_names.FirstName.notna()]).to_dict()
len(dict_names_doubles)

897

Use dictionary for dfauthors_full to add first names for the authors without them. The number of authors without full names is 6198.

In [302]:
df_authors_full['FirstName'][df_authors_full.FirstName.isnull()] = df_authors_full['Author'].map(dict_names_doubles)
len(df_authors_full['FirstName'][df_authors_full.FirstName.isnull()])

C:\Users\afedorova\AppData\Local\Temp\ipykernel_6064\4044942072.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_authors_full['FirstName'][df_authors_full.FirstName.isnull()] = df_authors_full['Author'].map(dict_names_doubles)


6198

### Add gender for the author, that were manually found

Create two dictionaries from csv with manually found first names and genders. Join them.

In [324]:
dict_Author_Gender = Series(df_mostPopularNames.Gender.values,index=df_mostPopularNames.Author).to_dict()
dict_Author_Gender2 = Series(df_mostPopularTitles.Gender.values,index=df_mostPopularTitles.Author).to_dict()
dict_Author_Gender.update(dict_Author_Gender2)
len(dict_Author_Gender)

1358

Use dictionary to add Gender to the main DataFrame

In [384]:
df_authors_full['Gender'] = df_authors_full['Author'].map(dict_Author_Gender)
df_authors_full.Gender = df_authors_full.Gender.str.lower()
df_authors_full.Gender.value_counts()

male      1075
female     804
Name: Gender, dtype: int64

### Create a batch for accuracy calculation from the names, that already have gender assigned.

In [385]:
unique_names_female = list(df_authors_full['Author'][df_authors_full['Gender'] == 'female'].unique())
unique_names_male = list(df_authors_full['Author'][df_authors_full['Gender'] == 'male'].unique())

In [394]:
import random

test_batch_fem = random.sample(unique_names_female, 300)
test_batch_m = random.sample(unique_names_male, 300)

test_batch = test_batch_fem + test_batch_m
test_batch_info = df_authors_full[['Author', 'FirstName', 'LastName','Gender', 'Title', 'URL']].loc[df_authors_full['Author'].isin(test_batch)]
test_batch_info.drop_duplicates(['Author'], inplace=True) 
test_batch_info
test_batch_info.to_csv('data/test_batch_full_info.csv')

In [395]:
test_batch_info

,Author,FirstName,LastName,Gender,Title,URL
15581,V. Brescoll,Victoria,Brescoll,female,Science faculty’s subtle gender biases favor m...,https://www.semanticscholar.org/paper/a7d48171...
15583,J. Handelsman,Jo,Handelsman,female,Science faculty’s subtle gender biases favor m...,https://www.semanticscholar.org/paper/a7d48171...
15607,T. Woodruff,Teresa,Woodruff,female,The Possible Role of Resource Requirements and...,https://www.semanticscholar.org/paper/20262c6d...
15729,V. Regitz-Zagrosek,Vera,Regitz-Zagrosek,female,Sex and gender differences in health,https://www.semanticscholar.org/paper/64c31156...
15745,P. McGuire,Philip,McGuire,male,Gender Differences in White Matter Microstructure,https://www.semanticscholar.org/paper/9e091f5f...
...,...,...,...,...,...,...
33951,C. Yong-Hing,Charlotte,Yong-Hing,female,Canadian Radiology Gender Pay Gap-Reality or M...,https://www.semanticscholar.org/paper/d080be3b...
34045,P. Zapater,Pedro,Zapater,male,Gender Biases and Diagnostic Delay in Inflamma...,https://www.semanticscholar.org/paper/3f06785e...
34047,C. van-der Hofstadt,Carlos,Hofstadt,male,Gender Biases and Diagnostic Delay in Inflamma...,https://www.semanticscholar.org/paper/3f06785e...
34267,B. Hanif,Bashir,Hanif,male,Gender Differences in Clinical Outcomes After ...,https://www.semanticscholar.org/paper/ee0f8bbe...


## Add and analyse Journal Names

In [129]:
df_TitleJournalName = read_csv('data/dict_TitleJournalName.csv',sep=';', names=['Title', 'JournalName'], error_bad_lines=False, index_col=False,encoding_errors='ignore')
df_TitleJournalName

C:\Users\afedorova\AppData\Local\Temp\ipykernel_4432\1525792281.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_TitleJournalName = read_csv('data/dict_TitleJournalName.csv',sep=';', names=['Title', 'JournalName'], error_bad_lines=False, index_col=False,encoding_errors='ignore')
Skipping line 13037: expected 2 fields, saw 4
Skipping line 15539: expected 2 fields, saw 3
Skipping line 15974: expected 2 fields, saw 4
Skipping line 17509: expected 2 fields, saw 4
Skipping line 18979: expected 2 fields, saw 4
Skipping line 32430: expected 2 fields, saw 3
Skipping line 41106: expected 2 fields, saw 3
Skipping line 41112: expected 2 fields, saw 3
Skipping line 41229: expected 2 fields, saw 3
Skipping line 41439: expected 2 fields, saw 4
Skipping line 42592: expected 2 fields, saw 3
Skipping line 46727: expected 2 fields, saw 3
Skipping line 50461: expected 2 fields, saw 4
Skipping line 52234: 

,Title,JournalName
0,Stag Parties Linger: Continued Gender Bias in ...,NaN
1,Gender Bias in the Iranian High School EFL Tex...,NaN
2,A Content Analysis of Textbooks: Investigating...,NaN
3,Reconsidering Gender Bias in Intrahousehold Al...,NaN
4,Gender Bias in Intrahousehold Allocation: Evid...,NaN
...,...,...
97848,"Land and Living on Little in Kenya,African Stu...",NaN
97849,An Overview of Eating Disorders in Iran: Frequ...,NaN
97850,The impact of riboflavin on the duration of sp...,NaN
97851,Uncovering the consequences of batch effect as...,NaN


In [130]:
df_TitleJournalName[df_TitleJournalName.JournalName.notnull()]

,Title,JournalName
130,Gender and Select Committee Elections in the B...,Gender
251,Cognitive Flexibility in ASD,"Task Switching with Emotional Faces,Journal o..."
285,Unexpected Winners: The Significance of an Ope...,Gender
455,Gender and Legislative Preferences: Evidence f...,Gender
471,"Gender, Race, Ethnicity, and Political Represe...",Gender
...,...,...
97636,News website users in Mexico: factors influenc...,Society
97677,Stereotyped gender roles and scripts in digita...,Media Studies
97714,Ten-Year Trends and Predictors of Unplanned Ho...,Social Care in the Community
97765,Abstract A123: Providing breast cancer screeni...,Prevention


In [33]:
df_authors_full['Journal'] = df_authors_full['Title'].map(dict_TitleJournalName)
df_authors_full[['Title', 'Journal']]
print(len(df_authors_full[df_authors_full.Journal.isna()]))

24454


### Create a column for paperID to be able to crawl additional information from SemanticScholar for specific papers

In [ ]:
def id_separator(paperURL):
    """Splits the paper URL into address and ID and saves ID
    
    :param: paper URL
    :return: paper ID
    """
    paperURL = str(paperURL)
    name = paperURL.rsplit('/', 1)[-1]
    return name

df_authors_full['PaperID']= df_authors_full.URL.apply(lambda x: id_separator(x))

### Crawl Journal names and Fields of Study

In [ ]:
sch = SemanticScholar(api_key=config.s2_api_key)
list_id = list(df_authors_full.PaperID[:74])

paper = sch.get_papers(list_id)
for item in paper:
     print(item.paperId, item.journal, item.fieldsOfStudy, item.s2FieldsOfStudy)

In [88]:
list_id = list(df_authors_full.PaperID.unique())
len(list_id)
additionalInfo = []

11544

In [162]:
sch = SemanticScholar() #api_key=config.s2_api_key

In [166]:
for i in range(len(additionalInfo)+1, len(list_id)):
    #list_i = list(df_authors_full.PaperID[i:(i+50)])
    item = sch.get_paper(list_id[i])
    additionalInfo.append({'PaperID': item['paperId'], 
        'Title': item['title'], 
        'FieldsOfStudy': item['fieldsOfStudy'],
        'FieldsOfStudy_s2': item['s2FieldsOfStudy'],
        'Journal': item['journal']
        })
    print(i, 'is ready')
    time.sleep(2)
    if i % 100 == 0:
        time.sleep(60)

11294 is ready
11295 is ready
11296 is ready
11297 is ready
11298 is ready
11299 is ready
11300 is ready
11301 is ready
11302 is ready
11303 is ready
11304 is ready
11305 is ready
11306 is ready
11307 is ready
11308 is ready
11309 is ready
11310 is ready
11311 is ready
11312 is ready
11313 is ready
11314 is ready
11315 is ready
11316 is ready
11317 is ready
11318 is ready
11319 is ready
11320 is ready
11321 is ready
11322 is ready
11323 is ready
11324 is ready
11325 is ready
11326 is ready
11327 is ready
11328 is ready
11329 is ready
11330 is ready
11331 is ready
11332 is ready
11333 is ready
11334 is ready
11335 is ready
11336 is ready
11337 is ready
11338 is ready
11339 is ready
11340 is ready
11341 is ready
11342 is ready
11343 is ready
11344 is ready
11345 is ready
11346 is ready
11347 is ready
11348 is ready
11349 is ready
11350 is ready
11351 is ready
11352 is ready
11353 is ready
11354 is ready
11355 is ready
11356 is ready
11357 is ready
11358 is ready
11359 is ready
11360 is r

batch request for 50 ids

In [ ]:
#additionalInfo = []
#list_id = list(df_authors_full.PaperID.unique())

for i in range(len(additionalInfo), len(list_id), 50):
    list_i = list(df_authors_full.PaperID[i:(i+50)])
    paper = sch.get_papers(list_i)
    for item in paper:
        additionalInfo.append({'PaperID': item['paperId'], 
            'Title': item['title'], 
            'FieldsOfStudy': item['fieldsOfStudy'],
            'FieldsOfStudy_s2': item['s2FieldsOfStudy'],
            'Journal': item['journal']
            })
    print(i+50, 'is ready')
    time.sleep(5)
    if i % 500 == 0:
        time.sleep(300)

In [167]:
df_add_info = DataFrame(additionalInfo)

In [168]:
df_add_info.to_csv('data/info_id_journal_filds.csv')

In [169]:
dict_IdJournal = {}
for data in additionalInfo:
    #print(data['Journal'].keys())
    try:
        dict_IdJournal[data['PaperID']] = data['Journal']['name']
    except:
        pass

df_authors_full['Journal'] = df_authors_full['PaperID'].map(dict_IdJournal)
df_authors_full['Journal'][df_authors_full.Journal.notnull()]

0               The Journal of Development Studies
1                     English Linguistics Research
2               Review of Economics and Statistics
3               Review of Economics and Statistics
4             North American Journal of Psychology
                           ...                    
41635        Journal of Cross-Cultural Gerontology
41636        Journal of Cross-Cultural Gerontology
41637    European Journal of Engineering Education
41638    European Journal of Engineering Education
41639    European Journal of Engineering Education
Name: Journal, Length: 37671, dtype: object

In [170]:
df_authors_full[df_authors_full.Journal.isna()]

,Title,Year,Author,PublicationType,CitationCount,URL,FirstName,LastName,MiddleName,PaperID,Journal
8,Socialisation and gender bias at the household...,2012,Suchismita Mishra,NaN,9,https://www.semanticscholar.org/paper/e3b54928...,Suchismita,Mishra,NaN,e3b54928abbc61c33aefdf21a89c3ab49ee101cc,NaN
34,What ’ s in a name ? A personnel selection exp...,2012,Christina Keinert-Kisin,NaN,2,https://www.semanticscholar.org/paper/650e60d9...,Christina,Keinert-Kisin,NaN,650e60d93323d8644390ce621c8d8ccac3ab9a28,NaN
48,Gender Bias in Reading and Related Literature ...,2012,Judith Zerring,['Review'],0,https://www.semanticscholar.org/paper/fd472b87...,Judith,Zerring,NaN,fd472b87d4c24e3e5c4b96d6a50caa32fc43847d,NaN
52,What are the underlying issues contributing to...,2012,Wanda Denise Carr,NaN,1,https://www.semanticscholar.org/paper/b5e69464...,Wanda,Carr,Denise,b5e6946403fc4528683cfb1450919b82b9de2f5e,NaN
67,Gender and Racial Bias in the Courtroom,2012,Ann T. Greeley,['Review'],0,https://www.semanticscholar.org/paper/5a275de6...,Ann,Greeley,T.,5a275de67eeb74f8c872da6e544c9c86e38c3627,NaN
...,...,...,...,...,...,...,...,...,...,...,...
41442,Gender inequality and not female mentors hinde...,2020,L. Diele-Viegas,NaN,2,https://www.semanticscholar.org/paper/69aa6f12...,Luisa,Diele-Viegas,NaN,69aa6f12ab8a78a8eb19a373f9450c012496bd5f,NaN
41443,Gender inequality and not female mentors hinde...,2020,C. Bacon,NaN,2,https://www.semanticscholar.org/paper/69aa6f12...,Christine,Bacon,NaN,69aa6f12ab8a78a8eb19a373f9450c012496bd5f,NaN
41622,A Statistical Inquiry into Gender-Based Income...,2022,A. Kaazempur-Mofrad,NaN,0,https://www.semanticscholar.org/paper/53cba202...,Ali,Kaazempur-Mofrad,NaN,53cba202823003c32ab973a523afa1699e67c884,NaN
41630,A review of gender inequality and women’s empo...,2023,R. Adam,['Review'],0,https://www.semanticscholar.org/paper/9e84c068...,Rahma,Adam,NaN,9e84c068b39ca7793f97ab5b05d11d6a58ec6056,NaN


In [111]:
dict_IdJournal

{'2062bd1530ba6df30912c78932534c99604ea039': 'Journal of Public Administration Research and Theory',
 '236d56dc3ac09b932ab8f828d2ad9be1868b8dc0': 'The Hearing Journal',
 '0d3d91f9dc528d0db6209c8cf9d62068addebdc6': 'Scientometrics',
 '4d838eed7a092528799f7abf60334ca817598f85': '',
 '147ba4663ce54eb8304ac9cf6acbe09f7db16e76': '',
 '55e4d488abde72de80376fb9cdc2b9c9474dd639': '',
 '8a5723307cd5e7b47aa922762b504934adce4f50': 'North American Journal of Psychology',
 'a8d29e1fec8a3019b82824d361f8cb65b7723ccc': '',
 'b2a1f91427fba3cbcbbf5487db100d3820d80c67': '',
 'b1732be98e58261a7e34f1c31e6a4109112c5e8c': 'Gaceta sanitaria',
 '21f199b4f6313391af46dc137a40038500f3e0de': 'Psychological assessment',
 'e8a39a2a06b21e86243a7229f47b5f7f4ead2c1f': '',
 'ed0129b344cd37a13bf6361bb270949993ab77f3': 'Hypertension',
 'eef46b8e89b5182fcd8e0168ba01bd6d3110f23e': '',
 '5ef099b67b64067cdddf0a3b6d9ceb2e4730ae80': 'Review of Economics and Statistics',
 '9bb8221b71a979b4d0d331db3d5d1f4091d54084': 'Br. J. Educ.

In [109]:
for data in additionalInfo:
    #print(data['Journal'].keys())
    print(data['PaperID'], data['Journal']['name'])

2062bd1530ba6df30912c78932534c99604ea039 Journal of Public Administration Research and Theory
236d56dc3ac09b932ab8f828d2ad9be1868b8dc0 The Hearing Journal
0d3d91f9dc528d0db6209c8cf9d62068addebdc6 Scientometrics
4d838eed7a092528799f7abf60334ca817598f85 
147ba4663ce54eb8304ac9cf6acbe09f7db16e76 
55e4d488abde72de80376fb9cdc2b9c9474dd639 
8a5723307cd5e7b47aa922762b504934adce4f50 North American Journal of Psychology
a8d29e1fec8a3019b82824d361f8cb65b7723ccc 
b2a1f91427fba3cbcbbf5487db100d3820d80c67 
b1732be98e58261a7e34f1c31e6a4109112c5e8c Gaceta sanitaria
21f199b4f6313391af46dc137a40038500f3e0de Psychological assessment


TypeError: 'NoneType' object is not subscriptable

In [161]:
len(additionalInfo)

7799

# Determine gender by name
## Genderize.io

In [ ]:
genders = []
for name in [name for name in test_batch_full_info.FirstName]:
    req = requests.get("https://api.genderize.io?name=" + name) #https://api.genderize.io?name=peter
    result = json.loads(req.text)
    genders.append(result)

print(genders)

In [417]:
batch_genderize = read_csv('data/test_batch_full_info_600_genderize.csv', index_col=[0]).reset_index(drop=True)
batch_genderize = batch_genderize.rename(columns={'gender':'gender_genderize'})
batch_genderize = batch_genderize.head(-1)

In [452]:
conf_mat = crosstab(batch_genderize['gender_genderize'], batch_genderize['Gender'])
print(conf_mat)

acc, prec, rec = calculate_metrics(batch_genderize['Gender'], batch_genderize['gender_genderize'])
print(acc, prec, rec)

Gender            female  male
gender_genderize              
female               278     7
male                  15   281
unknown                2     7
0.9522998296422487 0.9525423728813559 0.9429530201342282


In [443]:
def calculate_metrics(gender_true, gender_pred):
    conf_mat = crosstab(gender_pred, gender_true)
    if 'unknown' in conf_mat.index:
        TN = conf_mat['female']['female']
        TP = conf_mat['male']['male']
        FN = conf_mat['female']['male'] + conf_mat['female']['unknown']
        FP = conf_mat['male']['female'] + conf_mat['male']['unknown']
    else:
        TN = conf_mat['female']['female']
        TP = conf_mat['male']['male']
        FN = conf_mat['female']['male']
        FP = conf_mat['male']['female']

    acc = (TN + TP)/(TP + FP + TN + FP)
    prec = TP / (TP + FP)
    rec = TP / (TP + FN)
    return acc, prec, rec

## Gender API

In [ ]:
genderapi_url = 'https://gender-api.com/get?name='
genderapi_key = config.genderapi_api_key

genderapi = []

for name in [name for name in df_authors_full.FirstName.head()]:
    req = requests.get(genderapi_api+name+genderapi_key)
    result = json.loads(req.text)
    genderapi.append(result)

print(genderapi)

## Namsor

In [361]:
namsor_api_key = config_keys.namsor_api_key

namsor_url = "https://v2.namsor.com/NamSorAPIv2/api2/json/genderBatch"
FirstName = test_batch_info.FirstName.iloc[0]
LastName = test_batch_info.LastName.iloc[0]

namsor_req =  {
  "personalNames": [
    {
      "firstName": "Keith",
      "lastName": "Haring"
    }
  ]
}

headers = {
    "X-API-KEY": config_keys.namsor_api_key,
    "Accept": "application/json",
    "Content-Type": "application/json"
}

response = requests.request("POST", namsor_url, json=json.dumps(namsor_req), headers=headers)

print(response.text)

Cannot construct instance of `com.namsor.api2.model.in.BatchFirstLastNameIn` (although at least one Creator exists): no String-argument constructor/factory method to deserialize from String value ('{"personalNames": [{"firstName": "Keith", "lastName": "Haring"}]}')
 at [Source: (org.glassfish.jersey.message.internal.ReaderInterceptorExecutor$UnCloseableInputStream); line: 1, column: 1]


In [397]:
batch_namsor = read_csv('data/namsor_genderize-name_test_batch_full_info_600.csv', index_col=[0]).reset_index(drop=True)
batch_namsor

,Author,FirstName,LastName,Gender,Title,URL,apiVersion,script,id,firstName,lastName,likelyGender,genderScale,score,probabilityCalibrated
0,V. Brescoll,Victoria,Brescoll,female,Science faculty’s subtle gender biases favor m...,https://www.semanticscholar.org/paper/a7d48171...,namsor_api_2.0.24,LATIN,6491baae-b17d-4f7b-a264-12f11ef8de15,Victoria,Brescoll,female,0.983076,23.849708,0.991538
1,J. Handelsman,Jo,Handelsman,female,Science faculty’s subtle gender biases favor m...,https://www.semanticscholar.org/paper/a7d48171...,namsor_api_2.0.24,LATIN,fa327b23-1a27-4dde-977c-03b0ab28b69f,Jo,Handelsman,female,0.001884,2.036693,0.500942
2,T. Woodruff,Teresa,Woodruff,female,The Possible Role of Resource Requirements and...,https://www.semanticscholar.org/paper/20262c6d...,namsor_api_2.0.24,LATIN,7e545dd8-f6ac-4578-9b76-2b8de7ec877d,Teresa,Woodruff,female,0.994467,32.788609,0.997233
3,V. Regitz-Zagrosek,Vera,Regitz-Zagrosek,female,Sex and gender differences in health,https://www.semanticscholar.org/paper/64c31156...,namsor_api_2.0.24,LATIN,87a26705-3b17-4820-a6b2-8d96d06d8b7e,Vera,Regitz-Zagrosek,female,0.987570,26.470527,0.993785
4,P. McGuire,Philip,McGuire,male,Gender Differences in White Matter Microstructure,https://www.semanticscholar.org/paper/9e091f5f...,namsor_api_2.0.24,LATIN,35f3804d-1a73-4513-9363-b01eb92b2cf8,Philip,McGuire,male,-0.989370,27.390306,0.994685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,C. Yong-Hing,Charlotte,Yong-Hing,female,Canadian Radiology Gender Pay Gap-Reality or M...,https://www.semanticscholar.org/paper/d080be3b...,namsor_api_2.0.24,LATIN,dfc582cf-99f0-4689-9ef5-6608d69cc061,Charlotte,Yong-Hing,female,0.980044,22.719674,0.990022
596,P. Zapater,Pedro,Zapater,male,Gender Biases and Diagnostic Delay in Inflamma...,https://www.semanticscholar.org/paper/3f06785e...,namsor_api_2.0.24,LATIN,112c75f5-33dc-40c8-973b-0c2a6b50a47f,Pedro,Zapater,male,-0.994036,32.017052,0.997018
597,C. van-der Hofstadt,Carlos,Hofstadt,male,Gender Biases and Diagnostic Delay in Inflamma...,https://www.semanticscholar.org/paper/3f06785e...,namsor_api_2.0.24,LATIN,07b25baf-5269-4ebb-8eca-fa159d56b89b,Carlos,Hofstadt,male,-0.987188,26.185240,0.993594
598,B. Hanif,Bashir,Hanif,male,Gender Differences in Clinical Outcomes After ...,https://www.semanticscholar.org/paper/ee0f8bbe...,namsor_api_2.0.24,LATIN,f38decd9-fcfe-4b0f-8636-4648ccbc5a9d,Bashir,Hanif,male,-0.981743,23.253589,0.990872


Accuracy for batch with Namsor: 0.96

In [400]:
len(batch_namsor[batch_namsor['Gender'] == batch_namsor['likelyGender']]) / len(batch_namsor)

0.9616666666666667

In [399]:
crosstab(batch_namsor['Gender'], batch_namsor['likelyGender'])

likelyGender,female,male
Gender,,
female,287,13
male,10,290


In [404]:
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score

print(confusion_matrix(batch_namsor['Gender'], batch_namsor['likelyGender']))
print('Accuracy =', accuracy_score(batch_namsor['Gender'], batch_namsor['likelyGender']))
#print('F1 score', f1_score(batch_namsor['Gender'], batch_namsor['likelyGender']))
print(precision_score(batch_namsor['Gender'], batch_namsor['likelyGender']))

[[287  13]
 [ 10 290]]
Accuracy = 0.9616666666666667


ValueError: pos_label=1 is not a valid label. It should be one of ['female', 'male']

In [423]:
def calculate_metrics(TN, TP, FN, FP):
    acc = (TN + TP)/(TP + FP + TN + FP)
    prec = TP / (TP + FP)
    rec = TP / (TP + FN)
    return acc, prec, rec

TN = 287
TP = 290
FN = 10
FP = 13
acc, prec, rec = calculate_metrics(TN, TP, FN, FP)
print(acc, prec, rec)

0.956882255389718 0.9570957095709571 0.9666666666666667


In [ ]:
list_name_surname = list(zip([name for name in test_batch_info['FirstName']], [surname for surname in test_batch_info['LastName']]))
list_name_surname

## NameAPI

In [469]:
nameapi_url = "https://api.nameapi.org/rest/v5.3/genderizer/persongenderizer?apiKey="+nameapi_api_key

genders_nameapi = []

for name in list_name_surname:
    payload = {
        "context" : {
            "priority" : "REALTIME",
            "properties" : [ ]
        },
        "inputPerson": {
            "type": "NaturalInputPerson",
            "personName": {
                "nameFields": [
                    {
                        "string": name[0],
                        "fieldType": "GIVENNAME"
                    }, {
                        "string": name[1],
                        "fieldType": "SURNAME"
                    }
                ]
            },
            "gender": "UNKNOWN"
            }
        }
    try:
        resp = requests.post(nameapi_url, json=payload)
        resp.raise_for_status()
        resp_dict = resp.json()
        genders_nameapi.append({
            'FirstName': name[0],
            'LastName': name[1],
            'gender_nameapi': resp_dict['gender'].lower(),
            'confidence_nameapi': resp_dict['confidence']
        })
        #genders.append(result)
    except requests.exceptions.HTTPError as e:
        print("Bad HTTP status code:", e)
    except requests.exceptions.RequestException as e:
        print("Network error:", e)

Bad HTTP status code: 403 Client Error: AccessDenied for url: https://api.nameapi.org/rest/v5.3/genderizer/persongenderizer?apiKey=7738a52a43d336acbf1b6ae7375cc040-user1
Bad HTTP status code: 403 Client Error: AccessDenied for url: https://api.nameapi.org/rest/v5.3/genderizer/persongenderizer?apiKey=7738a52a43d336acbf1b6ae7375cc040-user1
Bad HTTP status code: 403 Client Error: AccessDenied for url: https://api.nameapi.org/rest/v5.3/genderizer/persongenderizer?apiKey=7738a52a43d336acbf1b6ae7375cc040-user1
Bad HTTP status code: 403 Client Error: AccessDenied for url: https://api.nameapi.org/rest/v5.3/genderizer/persongenderizer?apiKey=7738a52a43d336acbf1b6ae7375cc040-user1
Bad HTTP status code: 403 Client Error: AccessDenied for url: https://api.nameapi.org/rest/v5.3/genderizer/persongenderizer?apiKey=7738a52a43d336acbf1b6ae7375cc040-user1
Bad HTTP status code: 403 Client Error: AccessDenied for url: https://api.nameapi.org/rest/v5.3/genderizer/persongenderizer?apiKey=7738a52a43d336acbf1

KeyboardInterrupt: 

In [470]:
len(genders_nameapi)

430

In [473]:
batch_nameapi = DataFrame(genders_nameapi)
batch_nameapi['Gender'] = batch_namsor['Gender'].loc[:430]

In [474]:
batch_nameapi

,FirstName,LastName,gender_nameapi,confidence_nameapi,Gender
0,Victoria,Brescoll,female,0.955056,female
1,Jo,Handelsman,male,1.000000,female
2,Teresa,Woodruff,female,0.897635,female
3,Vera,Regitz-Zagrosek,female,0.955056,female
4,Philip,McGuire,male,0.933333,male
...,...,...,...,...,...
425,Metin,Akgun,male,0.913679,male
426,Barbara,Borg,female,0.944712,female
427,Alex,Mullen,neutral,1.000000,female
428,Silvia,Orlandi,female,0.955056,female


In [475]:
calculate_metrics(batch_nameapi['gender_nameapi'], batch_nameapi['Gender'])

(0.94, 0.9408866995073891, 0.9744897959183674)

# In case of emergency

In [2]:
df_authors_full = read_csv('data/df_authors_full.csv', index_col=False)
df_authors_full.Year = df_authors_full.Year.astype('Int64')
df_authors_full.CitationCount = df_authors_full.CitationCount.astype('Int64')
df_authors_full = df_authors_full.iloc[: , 1:]
df_authors_full.columns

Index(['Title', 'Year', 'Author', 'PublicationType', 'CitationCount', 'URL',
       'FirstName', 'LastName', 'MiddleName'],
      dtype='object')

In [3]:
len(df_authors_full)

41640

In [6]:
df_authors_full.Title = df_authors_full.Title.str.rstrip('.')

In [4]:
df_authors_full[['Title', 'Author', 'FirstName']][df_authors_full.Author == 'M. LaFrance']

,Title,Author,FirstName
27101,Gender Bias in Pediatric Pain Assessment,M. LaFrance,Marianne
28803,Featured Article: Gender Bias in Pediatric Pai...,M. LaFrance,NaN
35991,"""Master"" of none: Institutional language chang...",M. LaFrance,Marianne
36130,"Gendered Anti-Bisexual Bias: Heterosexual, Bis...",M. LaFrance,Marianne


In [5]:
index = df_authors_full[['Title', 'Author', 'FirstName']][df_authors_full.duplicated(['Title', 'Author', 'FirstName'])].index #[df_authors_full.FirstName.isnull()].sort_values('Title')
print(len(index))
df_authors_full = df_authors_full.drop(index)

0


In [10]:
index2 = df_authors_full[df_authors_full.duplicated(['Title', 'Author'], keep=False)][df_authors_full.FirstName.isnull()].index
print(len(index2))
df_authors_full = df_authors_full.drop(index2)

20


C:\Users\afedorova\AppData\Local\Temp\ipykernel_17364\977037083.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  index2 = df_authors_full[df_authors_full.duplicated(['Title', 'Author'], keep=False)][df_authors_full.FirstName.isnull()].index


In [ ]:
df_authors_full.head(10)

In [21]:
df_authors_full.to_csv('data/df_authors_full.csv')

In [31]:
def id_separator(paperURL):
    """Splits the paper URL into address and ID and saves ID
    
    :param: paper URL
    :return: paper ID
    """
    paperURL = str(paperURL)
    name = paperURL.rsplit('/', 1)[-1]
    return name

In [20]:
type(df_authors_full.URL[0])

str

In [34]:
df_authors_full['PaperID']= df_authors_full.URL.apply(lambda x: id_separator(x))

,URL,PaperID
0,https://www.semanticscholar.org/paper/0aa45744...,0aa45744fa78e2c4675c82a76e9619f38cc31f92
1,https://www.semanticscholar.org/paper/2045f3d1...,2045f3d1c6ef3d3c91a8346c54f7d428df524cc3
2,https://www.semanticscholar.org/paper/5ef099b6...,5ef099b67b64067cdddf0a3b6d9ceb2e4730ae80
3,https://www.semanticscholar.org/paper/5ef099b6...,5ef099b67b64067cdddf0a3b6d9ceb2e4730ae80
4,https://www.semanticscholar.org/paper/8a572330...,8a5723307cd5e7b47aa922762b504934adce4f50
...,...,...
41635,https://www.semanticscholar.org/paper/af4482b5...,af4482b5f05e7d900885a1d3f46227c076df5edb
41636,https://www.semanticscholar.org/paper/af4482b5...,af4482b5f05e7d900885a1d3f46227c076df5edb
41637,https://www.semanticscholar.org/paper/f81610d8...,f81610d893e008d4b5646c02128a293baa1f8e9b
41638,https://www.semanticscholar.org/paper/f81610d8...,f81610d893e008d4b5646c02128a293baa1f8e9b


## Manual update

In [2]:
'R. Dietz': 'Richard Dietz',
'R. Pearson':  'Richard Pearson',
'M. Semak': 'Matthew R Semak',
'C. Willis': 'Courtney W Willis',
'D. Syed': 'Deeba N. Syed',
'H. Mukhtar': 'Hasan Mukhtar'

SyntaxError: illegal target for annotation (1957024364.py, line 1)

How is the relative distribution of gender among the researchers, who study gender bias?

How is the temporal dynamics of the female representation as well as the decade forecast?

Check search pattern "women + bias" (as in "bias against women") in df_authors, that are not in df_authors_re

try google scholar with SemanticScholar for author names